In [11]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('../data/loan_data.csv')

In [6]:
df.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


## Prepare data

In [13]:
# Automatically encode all object/categorical columns
df_encoded = pd.get_dummies(df)

In [14]:
df_encoded.head()

,person_age,person_income,person_emp_exp,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,loan_status,person_gender_female,...,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,previous_loan_defaults_on_file_No,previous_loan_defaults_on_file_Yes
0,22.0,71948.0,0,35000.0,16.02,0.49,3.0,561,1,True,...,False,True,False,False,False,False,True,False,True,False
1,21.0,12282.0,0,1000.0,11.14,0.08,2.0,504,0,True,...,True,False,False,True,False,False,False,False,False,True
2,25.0,12438.0,3,5500.0,12.87,0.44,3.0,635,1,True,...,False,False,False,False,False,True,False,False,True,False
3,23.0,79753.0,0,35000.0,15.23,0.44,2.0,675,1,True,...,False,True,False,False,False,True,False,False,True,False
4,24.0,66135.0,1,35000.0,14.27,0.53,4.0,586,1,False,...,False,True,False,False,False,True,False,False,True,False


## Determine feature importance

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [18]:
X = df_encoded.drop('loan_status', axis=1)
y = df_encoded['loan_status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [21]:
# Train random forest and get feature importances
model = RandomForestClassifier()
model.fit(X_train, y_train)
importances = model.feature_importances_

In [22]:
# Display feature importances
feature_importances = pd.Series(importances, index=X_train.columns)
print(feature_importances.sort_values(ascending=False))

loan_percent_income                   0.151548
loan_int_rate                         0.145919
previous_loan_defaults_on_file_Yes    0.134159
previous_loan_defaults_on_file_No     0.122625
person_income                         0.114104
loan_amnt                             0.055886
credit_score                          0.054371
person_home_ownership_RENT            0.034050
person_age                            0.029296
person_emp_exp                        0.027237
cb_person_cred_hist_length            0.024824
person_home_ownership_MORTGAGE        0.021927
person_home_ownership_OWN             0.009251
loan_intent_HOMEIMPROVEMENT           0.008070
loan_intent_MEDICAL                   0.007430
loan_intent_DEBTCONSOLIDATION         0.007113
loan_intent_VENTURE                   0.006959
loan_intent_EDUCATION                 0.005960
person_education_Bachelor             0.005783
person_gender_male                    0.005682
person_education_High School          0.005582
person_educat

In [28]:
X_train = X_train[[
    'loan_percent_income', 
    'loan_int_rate', 
    'previous_loan_defaults_on_file_Yes', 
    'previous_loan_defaults_on_file_No', 
    'person_income'
]]
X_test = X_test[[
    'loan_percent_income', 
    'loan_int_rate', 
    'previous_loan_defaults_on_file_Yes', 
    'previous_loan_defaults_on_file_No', 
    'person_income'
]]

In [32]:
X_train.head()

,loan_percent_income,loan_int_rate,previous_loan_defaults_on_file_Yes,previous_loan_defaults_on_file_No,person_income
25180,0.15,12.73,False,True,97265.0
12555,0.16,11.86,True,False,72953.0
29153,0.07,10.37,True,False,322597.0
23838,0.10,17.14,False,True,94232.0
35686,0.08,12.97,True,False,84873.0


In [33]:
X_test.head()

,loan_percent_income,loan_int_rate,previous_loan_defaults_on_file_Yes,previous_loan_defaults_on_file_No,person_income
37979,0.08,6.04,False,True,96865.0
9911,0.16,11.49,True,False,56838.0
43386,0.13,14.88,False,True,37298.0
13822,0.13,13.99,True,False,39944.0
44810,0.15,15.41,False,True,67974.0


In [34]:
y_test.head()

37979    0
9911     0
43386    1
13822    0
44810    1
Name: loan_status, dtype: int64

## Train model

### Logistic Regression

In [30]:
from sklearn.linear_model import LogisticRegression
from joblib import dump, load

In [29]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

/Users/vincentcheng/Documents/data_science/loan-prediction-app-cicd/myenv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8591111111111112

In [31]:
# Save the model
dump(lr, '../models/lr_model.joblib')

['../models/lr_model.joblib']